In [18]:
'''
1. set up temp repo for rna seq data: /storage2/meta/rnaSeqData

2. write code to make rna-seq script automatically given the name of the downlaoded rna-seq file

3. make table in the format shortName, RNASeqFileName, downloadURL

4. the intronDensity VS gc plot is divided in to 3x3 blocks

'''
import pandas as pd
info = pd.read_csv("/home/richard/research/1_DataSmall/2_completeFileInfo/completeInfoStatsRNASeqFiltered.csv")
rnaSeqSource = '/storage3/w/richard/meta2015'

In [19]:
# tupleFormat: shortName, filename, URL
RNASeqFileInfo = []

In [38]:
## (1,1) low gc, high intron density
# print info.loc[139]

target = info.loc[139]
shortName = target['shortName']
fastaName = target['fastaName']

URL = "ftp://ftp-trace.ncbi.nlm.nih.gov/sra/sra-instant/reads/ByRun/sra/ERR/ERR135/ERR135878/ERR135878.sra"
fileName = 'ERR135878.sra'
RNASeqFileInfo.append((shortName, fileName, URL))

# make script for the whole process of rna seq, prep -> ran-seq
# create directory
script = 'cd {}/rnaSeqPlace\n'.format(rnaSeqSource)
script += 'mkdir {0}\ncd {0}\n'.format(shortName)

# prep folders
script += 'mkdir arx data data/starg star\n'

# copy genome file and unzip it
script += 'cp /home/richard/1_DataLarge/fasta_gff_gtf_07302015/{}/{} ./data\n'.format(shortName, fastaName)
script += 'gunzip ./data/{}\n'.format(fastaName)

# get RNA-seq 
script += 'cd arx\n'
script += 'cp {}/rnaSeqData/{} .\n'.format(rnaSeqSource,fileName)
script += '/home/tool/sratoolkit/bin/fastq-dump --split-3 {}\n'.format(fileName)


# prepare genome for star
script += 'cd ../data/starg\n'
script += '/home/tool/STAR/source/STAR --runMode genomeGenerate --genomeDir . --genomeFastaFiles ../{} --runThreadN 8\n'.format(fastaName[:-3])

## run rna-seq
# execute aux program and generate run_star
script += "cd ../../star\n"
script += 'python {} .\n'.format("/home/richard/research/4_SelfMadeTool/generateRunStar.py")
# execute run_star.sh
script += 'bash run_star.sh &\n'

with open("/home/richard/research/tempFile/rnaSeq_{}.sh".format(shortName), 'w') as f: f.write(script)

In [39]:
print script

cd /storage3/w/richard/meta2015/rnaSeqPlace
mkdir Pneji1
cd Pneji1
mkdir arx data data/starg star
cp /home/richard/1_DataLarge/fasta_gff_gtf_07302015/Pneji1/Pneji1_AssemblyScaffolds.fasta.gz ./data
gunzip ./data/Pneji1_AssemblyScaffolds.fasta.gz
cd arx
cp /storage3/w/richard/meta2015/rnaSeqData/ERR135878.sra .
/home/tool/sratoolkit/bin/fastq-dump --split-3 ERR135878.sra
cd ../data/starg
/home/tool/STAR/source/STAR --runMode genomeGenerate --genomeDir . --genomeFastaFiles ../Pneji1_AssemblyScaffolds.fasta --runThreadN 8
cd ../../star
python /home/richard/research/4_SelfMadeTool/generateRunStar.py .
bash run_star.sh &

